# **LP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. """

In [6]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

Let's denote:

1) X1 - the amount of GCA produced.
2) X2 - the amount of GCB produced.
3) X3 - the amount of GCC produced.
4) Y1 - the amount of C1 used in the production of GCA.
5) Y2 - the amount of C1 used in the production of GCB.
6) Y3 - the amount of C1 used in the production of GCC.
7) Z1 - the amount of C2 used in the production of GCA.
8) Z2 - the amount of C2 used in the production of GCB.
9) Z3 - the amount of C2 used in the production of GCC.


In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [9]:
print(response2.choices[0].message.content)

The objective function to maximize the profit can be formulated as follows:

Maximize P = 120*X1 + 135*X2 + 155*X3


### **Generate Pyomo Code**

In [10]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [11]:
print(response3.choices[0].message.content)

The constraints for this mathematical optimization problem can be formulated as follows:

1) X1 >= 6000 (Minimum demand level for GCA)
2) X2 >= 7000 (Minimum demand level for GCB)
3) X3 >= 9000 (Minimum demand level for GCC)
4) Y1 + Y2 + Y3 <= 10000 (Available quantity of C1)
5) Z1 + Z2 + Z3 <= 15000 (Available quantity of C2)
6) 0.4*Y1 + 0.2*Z1 >= 0.3*X1 (Minimum critical element fraction for GCA)
7) 0.4*Y2 + 0.2*Z2 <= 0.3*X2 (Maximum critical element fraction for GCB)
8) Y3/(Y3 + Z3) >= 0.3 (Minimum C1/C2 ratio in GCC)
9) X1 = Y1 + Z1 (No loss or gain in volume for GCA)
10) X2 = Y2 + Z2 (No loss or gain in volume for GCB)
11) X3 = Y3 + Z3 (No loss or gain in volume for GCC)
12) All variables are non-negative (X1, X2, X3, Y1, Y2, Y3, Z1, Z2, Z3 >= 0)


In [12]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [13]:
Markdown(response4.choices[0].message.content)

The corresponding python pyomo code is written as follows.

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.X3 = Var(within=NonNegativeReals)
model.Y1 = Var(within=NonNegativeReals)
model.Y2 = Var(within=NonNegativeReals)
model.Y3 = Var(within=NonNegativeReals)
model.Z1 = Var(within=NonNegativeReals)
model.Z2 = Var(within=NonNegativeReals)
model.Z3 = Var(within=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr = 120*model.X1 + 135*model.X2 + 155*model.X3, sense=maximize)

# Define constraints
model.con1 = Constraint(expr=model.X1 >= 6000)
model.con2 = Constraint(expr=model.X2 >= 7000)
model.con3 = Constraint(expr=model.X3 >= 9000)
model.con4 = Constraint(expr=model.Y1 + model.Y2 + model.Y3 <= 10000)
model.con5 = Constraint(expr=model.Z1 + model.Z2 + model.Z3 <= 15000)
model.con6 = Constraint(expr=0.4*model.Y1 + 0.2*model.Z1 >= 0.3*model.X1)
model.con7 = Constraint(expr=0.4*model.Y2 + 0.2*model.Z2 <= 0.3*model.X2)
model.con8 = Constraint(expr=model.Y3/(model.Y3 + model.Z3) >= 0.3)
model.con9 = Constraint(expr=model.X1 == model.Y1 + model.Z1)
model.con10 = Constraint(expr=model.X2 == model.Y2 + model.Z2)
model.con11 = Constraint(expr=model.X3 == model.Y3 + model.Z3)

# Solve
SolverFactory('glpk').solve(model).write()
```

In this code, we initially import the necessary modules from the Pyomo package. Then, a model is created and the variables (X1, X2, X3, Y1, Y2, Y3, Z1, Z2, Z3) are defined within the non-negative real numbers (since all the variables cannot take negative values).  The Objective function is defined to maximize the profit. The constraints are then defined using the constraint expression (`expr`) and passed into the `Constraint()` function. Finally, the model is solved using the 'glpk' solver and the results are printed.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.X3 = Var(within=NonNegativeReals)
model.Y1 = Var(within=NonNegativeReals)
model.Y2 = Var(within=NonNegativeReals)
model.Y3 = Var(within=NonNegativeReals)
model.Z1 = Var(within=NonNegativeReals)
model.Z2 = Var(within=NonNegativeReals)
model.Z3 = Var(within=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr = 120*model.X1 + 135*model.X2 + 155*model.X3, sense=maximize)

# Define constraints
model.con1 = Constraint(expr=model.X1 >= 6000)
model.con2 = Constraint(expr=model.X2 >= 7000)
model.con3 = Constraint(expr=model.X3 >= 9000)
model.con4 = Constraint(expr=model.Y1 + model.Y2 + model.Y3 <= 10000)
model.con5 = Constraint(expr=model.Z1 + model.Z2 + model.Z3 <= 15000)
model.con6 = Constraint(expr=0.4*model.Y1 + 0.2*model.Z1 >= 0.3*model.X1)
model.con7 = Constraint(expr=0.4*model.Y2 + 0.2*model.Z2 <= 0.3*model.X2)
model.con8 = Constraint(expr=model.Y3/(model.Y3 + model.Z3) >= 0.3)
model.con9 = Constraint(expr=model.X1 == model.Y1 + model.Z1)
model.con10 = Constraint(expr=model.X2 == model.Y2 + model.Z2)
model.con11 = Constraint(expr=model.X3 == model.Y3 + model.Z3)

# Solve
SolverFactory('glpk').solve(model).write()

ValueError: Model constraint (con8) contains nonlinear terms that cannot be written to LP format

In [20]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.X3 = Var(within=NonNegativeReals)
model.Y1 = Var(within=NonNegativeReals)
model.Y2 = Var(within=NonNegativeReals)
model.Y3 = Var(within=NonNegativeReals)
model.Z1 = Var(within=NonNegativeReals)
model.Z2 = Var(within=NonNegativeReals)
model.Z3 = Var(within=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr = 120*model.X1 + 135*model.X2 + 155*model.X3, sense=maximize)

# Define constraints
model.con1 = Constraint(expr=model.X1 >= 6000)
model.con2 = Constraint(expr=model.X2 >= 7000)
model.con3 = Constraint(expr=model.X3 >= 9000)
model.con4 = Constraint(expr=model.Y1 + model.Y2 + model.Y3 <= 10000)
model.con5 = Constraint(expr=model.Z1 + model.Z2 + model.Z3 <= 15000)
model.con6 = Constraint(expr=0.4*model.Y1 + 0.2*model.Z1 >= 0.3*model.X1)
model.con7 = Constraint(expr=0.4*model.Y2 + 0.2*model.Z2 <= 0.3*model.X2)
model.con8 = Constraint(expr=model.Y3 >= 0.3*(model.Y3 + model.Z3))
model.con9 = Constraint(expr=model.X1 == model.Y1 + model.Z1)
model.con10 = Constraint(expr=model.X2 == model.Y2 + model.Z2)
model.con11 = Constraint(expr=model.X3 == model.Y3 + model.Z3)

# Solve
SolverFactory('glpk').solve(model).write()
print(model.obj(), model.Y1(), model.Y2(),model.Y3(), model.Z1(), model.Z2(), model.Z3())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3525000.0
  Upper bound: 3525000.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 9
  Number of nonzeros: 26
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.029981374740600586
# ----------------------------------------------------------
#   Solution Information
# --------------------------

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**